<a href="https://colab.research.google.com/github/hy30n80/kaggle/blob/main/CJ_Kaggle_by_DART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Kaggle 데이터 불러오기

In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hy30n80","key":"fd641521b37ab39e25b1af679f36e7a4"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c store-sales-time-series-forecasting

 94% 20.0M/21.4M [00:00<00:00, 72.0MB/s]
100% 21.4M/21.4M [00:00<00:00, 68.0MB/s]


In [5]:
!unzip -qq "/content/store-sales-time-series-forecasting.zip" -d ./input

#1. 데이터 불러오기

In [6]:
import os
for dirname, _, filenames in os.walk('/content/input'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/input/stores.csv
/content/input/test.csv
/content/input/train.csv
/content/input/holidays_events.csv
/content/input/oil.csv
/content/input/sample_submission.csv
/content/input/transactions.csv


In [7]:
import pandas as pd

df_train = pd.read_csv('/content/input/train.csv')
display(df_train.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


**<train.csv>**
- id : row 의 index \
- date : 현재 날짜 \
- store_nbr : 가게 번호 \
- family : 물건 품목 \
- sales : 해당 품목에 대한 판매량 \
- onpromotion : 해당 품목에서 행사중인 상품의 수 \

In [8]:
df_holidays_events = pd.read_csv('/content/input/holidays_events.csv')
display(df_holidays_events.head())

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


**<holidays_events.csv>**
- date : 공휴일 날짜  
- type : 휴일 종류 (Holiday, Event, Tranfer, Additional, Bridge, Work Day) 
- locale : 행사 범위 (Local, Regional, National)  
- locale_name : 행사가 일어나는 지역 
- transferred : 행사가 다른 날로 이전되었는지, 아닌지


In [9]:
df_oil = pd.read_csv('/content/input/oil.csv')
display(df_oil.head())

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


**<oil.csv>** \
: 유가 

In [10]:
df_stores = pd.read_csv('/content/input/stores.csv')
display(df_stores.head())

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


**<stores.csv>** \
cluster : 근처에 있는 store 개수 \
type : Store 의 type

In [11]:
df_transactions = pd.read_csv('/content/input/transactions.csv')
display(df_transactions.head())

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


**<transactions.csv>**\
: 구매자의 영수증 수 (물품 산 개수 상관없이)

In [12]:
df_test = pd.read_csv('/content/input/test.csv')
df_sample_submission = pd.read_csv('/content/input/sample_submission.csv')
display(df_test.head())
display(df_sample_submission.head())

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


**<test.csv & sample_submission.csv>** \
: date, store_nbr, family, onpromotion 주고 예측

#2. 데이터 전처리

In [13]:
#데이터에 제품 품목, 가게 종류가 얼마나 있는지 확인
family_list = df_train['family'].unique()
store_list = df_stores['store_nbr'].unique()
display(family_list)
display(store_list)

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54])

In [14]:
#df_train 과 df_stores 을 합칠거야 
#기존에 train set에 가게 숫자만 있었다면, 이제는 해당 가게의 특성에 대한 feature 도 한번에 볼 수 있는거지

#store_nbr 을 기준으로 하나로 합침
train_merged = pd.merge(df_train, df_stores, on='store_nbr')

#store number, family, date 순으로 정렬
train_merged = train_merged.sort_values(['store_nbr','family', 'date'])

train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str', 
                                    "state":'str', "type":'str', "cluster":'str'})
#가게별 품목별, Serial data 로 전환된 모습
display(train_merged.head())

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
33,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13
66,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
99,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
132,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13


In [15]:
#test set 도 위처럼, 가게별 품몰별 serial data 형태로 정렬 -> 예측 하기 훨씬 수월해짐
df_test_dropped = df_test.drop(['onpromotion'], axis=1)
df_test_sorted = df_test_dropped.sort_values(by=['store_nbr','family'])

display(df_test_sorted.head())

,id,date,store_nbr,family
0,3000888,2017-08-16,1,AUTOMOTIVE
1782,3002670,2017-08-17,1,AUTOMOTIVE
3564,3004452,2017-08-18,1,AUTOMOTIVE
5346,3006234,2017-08-19,1,AUTOMOTIVE
7128,3008016,2017-08-20,1,AUTOMOTIVE


#3. Dart 도입

In [16]:
!pip install darts==0.23.1 &> /dev/null

목표 : 54개의 가게의 33개의 품목에 대한 각각의 Serial Data 를 예측해야 하는 것. => 우리는 총 1782개의 time series set을 갖고 있음 

1. 첫번째 방법 : 1782개의 time series model 을 학습 시켜서 각각 예측하는 것 -> 일반화하기 힘들다는 단점 & 학습 데이터 부족으로 정확도 떨어짐

2. 두번째 방법 : 하나의 time series 모델로 학습시키는 것 -> 정확도 떨어짐

3. 최종 선택 방법 : 54(가게 수)개의 time-series (시계열) 로 학습된 33개(품목 수)의 모델을 사용할 거야


***최종 : 각 품목 별로 예측 모델이 하나씩 나오는 거지***

In [17]:
train_merged.loc[train_merged['family']=='AUTOMOTIVE']

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
33,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13
66,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
99,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
132,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
2778435,2993595,2017-08-11,54,AUTOMOTIVE,11.0,0,El Carmen,Manabi,C,3
2778468,2995377,2017-08-12,54,AUTOMOTIVE,10.0,0,El Carmen,Manabi,C,3
2778501,2997159,2017-08-13,54,AUTOMOTIVE,16.0,0,El Carmen,Manabi,C,3
2778534,2998941,2017-08-14,54,AUTOMOTIVE,4.0,0,El Carmen,Manabi,C,3


In [18]:
import numpy as np
import darts
from darts import TimeSeries

family_TS_dict = {}

for family in family_list:

  #특정 품목 (예 "AUTOMOTIVE") 인 것들의 Dateframe 만 뽑아오기 
  df_family = train_merged.loc[train_merged['family']==family]

  #Timeseries.from_group_dataframe : Dataframe -> Timeseries List 로 변환
  list_of_TS_family = TimeSeries.from_group_dataframe(
      #df_family : 위 셀에서 출력한, family 별로 뽑은 dataframe
      df_family,
      time_col="date",
      #group_cols : 
      group_cols=["store_nbr","family"],
      #static_cols
      static_cols=["city","state","type","cluster"],
      value_cols="sales",
      #데이터셋에 12월 25일 값이 사라져있음
      fill_missing_dates=True,
      #시계열 데이터 간격이 Day 라는 뜻
      freq='D')
  for ts in list_of_TS_family:
    ts = ts.astype(np.float32)

  list_of_TS_family = sorted(list_of_TS_family, key=lambda ts: int(ts.static_covariates_values()[0,0]))
  #family_TS_dict 의 Key 는 family(품목)
  family_TS_dict[family] = list_of_TS_family



- **value_cols (Union[str, List[str], None])** – A string or list of strings representing the value column(s) to be extracted from the DataFrame. If set to None, the whole DataFrame will be used.

- **group_cols (Union[List[str], str])** – A string or list of strings representing the columns from the Data Frame by which to extract the individual TimeSeries groups. : 두개의 feature 를 하나로 생각한다는 것.

- **static_cols (Union[str, List[str], None])** – A string or list of strings representing static variable columns from the DataFrame that should be appended as static covariates to the resulting TimeSeries groups. Different to group_cols, the DataFrame is not grouped by these columns. Note that for every group, there must be exactly one unique value.

-> 각 Family 별로 timeseries 만들어 합쳐서, 딕셔너리 형태로 family_TS_dict 에 저장

In [19]:
display(type(list_of_TS_family))
display(type(family_TS_dict))

list

dict



> For each product family, we will gather all the time series concerning it.

> So we will have 33 sub-datasets. These datasets will be contained in the family_TS_dict dictionary. 

> In the following lines of code, we extract the TimeSeries of the 54 stores for each family.



In [20]:
#데이터 1688개 = 5년치 시계열, 밑에 명령어는 특정 store 의 AUTOMATIVE 시계열 데이터가 되겠군
#AUTOMOTIVE[0]~AUTOMOTIVE[53] 총 54개의 store 의 sales 판매량
display(family_TS_dict['AUTOMOTIVE'][53])

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)>
array([[[ 0.]],

       [[ 5.]],

       [[ 3.]],

       ...,

       [[16.]],

       [[ 4.]],

       [[ 8.]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates store_nbr      family       city   ...
    hierarchy:          None



> **static covariates : 시간의 흐름과 관계 없는 값들 : store_nbr, family, city, state, type, cluser -> 즉 아까 위에서 Timeseries 만들 때 'value_cols=sales', 'time_col=date" 을 제외한 모든 feature 들이 자동적으로 할당됨**


##Nomralizaing time series

- Scaler function of Dart 를 이용하여, 쉽게 Timeseries 를 normalizing 할 수 있음
- 더 나아가, 위의 covariates 들을 one-hot 코딩할거야 by StaticCovariatesTransformer function



> Pipeline 과정 : train_filler -> static_cov_transformer -> log_transformer -> train_scaler




> 결측값 처리 -> static-covariate 들 원핫 인코딩 -> log 함수 때려서 값 정규화 -> 정규화된 값들 (0,1) 사이로 들어오게 추가 스케일링 (참고로, static covariate 들은 안 건든다.)





In [21]:
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
import sklearn

family_pipeline_dict = {}
family_TS_transformed_dict = {}

#제품 카테고리들이 나오겠지 
for key in family_TS_dict:
  #MissingValuesFiller : 결측값 처리 라이브러리, n_jobs : thread 몇개로 돌릴지? , verbose : 과정 상세 설명 할지 
  train_filler = MissingValuesFiller(verbose=False, n_jobs=1, name="Fill NAs")
  #StaticCovariatesTransformer : covariates (features) 들 one-hot encoding
  static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder")
  #InvertibleMapper : 값들을 바꿔주는 함수 정도 생각 (예를 들어, 모든 값들에 같은 함수 취해서 값을 normalize 하거나) - 나중에 값을 돌릴수도 있어서 invertible
  #fn : 정규화 함수, inverse_fn : 정규화된 값 원래로 복귀 (나중에 결과값 돌려야지)
  log_transformer = InvertibleMapper(fn = np.log1p, inverse_fn = np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")   
  #Scaler : Normalize (0,1) 사이로 값을 집어넣는다는 등의. 참고로, The scaler will not scale the series’ static covariates (우리가 원핫인코딩 한거는 안건드린다)
  train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")


  train_pipeline = Pipeline([train_filler,
                              static_cov_transformer,
                              log_transformer,
                              train_scaler])
 
  training_transformed = train_pipeline.fit_transform(family_TS_dict[key])
  #각 제품 카테고리별 파이프라인 과정
  family_pipeline_dict[key] = train_pipeline
  #transformed 된 시계열 데이터 값들 (family_TS_dict -> family_TS_transformed_dict )
  family_TS_transformed_dict[key] = training_transformed

In [22]:
display(family_TS_transformed_dict['AUTOMOTIVE'][0])

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)>
array([[[0.        ]],

       [[0.36672579]],

       [[0.46275643]],

       ...,

       [[0.23137821]],

       [[0.23137821]],

       [[0.53724357]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr_1  store_nbr_10  store_n...
    hierarchy:          None

> **보면, sales 는 normalized 되었고, static_covariates 로 설정한 feature (store_nbr, family, city, state, type, cluser)  들은 one-hot encoding 된 모습!!**

> 지금 보면, ["city","state","type","cluster"] 에 대해서만 one-hot encoding 했고, 나머지 covariates 들은 데이터데 넣지도 않았는데 그 이유는 우리 다른 covariates (oil price, transaction) 들은 과거 지표이기 때문 => 우리가 예측할 미래의 test set 에는 oil price, transaction 주어지지 않기 때문이야 

##날짜를 찢어줍시다.
- year – year
- month – month
- day – day
- dayofyear – day of the year (for example February 1 is the 32nd day of the year)
- weekday – day of the week (there are 7 days in a week)
- weekofyear – week of the year (there are 52 weeks in a year)
- linear_increase – the index of the interval

In [23]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')

year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

In [24]:
print(full_time_period)

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08',
               '2013-01-09', '2013-01-10',
               ...
               '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25',
               '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29',
               '2017-08-30', '2017-08-31'],
              dtype='datetime64[ns]', name='time', length=1704, freq='D')


In [25]:
#날짜별로 연(year), 달(month), 날짜(date), 몇번째날(dayofyear), 몇번째요일(weekday), 몇번째주(weekofyear), 데이터 인덱스 = 몇번째 데이터인지 (timesteps)
#(2013, 01, 09, 09, 02(화?), 02, 08)
print(time_cov[8])

<TimeSeries (DataArray) (time: 1, component: 7, sample: 1)>
array([[[2.013e+03],
        [1.000e+00],
        [9.000e+00],
        [9.000e+00],
        [2.000e+00],
        [2.000e+00],
        [8.000e+00]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2013-01-09
  * component  (component) object 'year' 'month' ... 'linear_increase'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None


In [26]:
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

In [27]:
print(time_cov_transformed)

<TimeSeries (DataArray) (time: 1704, component: 7, sample: 1)>
array([[[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        ...,
        [1.6666667e-01],
        [0.0000000e+00],
        [0.0000000e+00]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [3.3333335e-02],
        ...,
        [3.3333334e-01],
        [0.0000000e+00],
        [5.9276825e-04]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [6.6666678e-02],
        ...,
...
        ...,
        [1.6666667e-01],
        [6.5384614e-01],
        [1.0082988e+00]],

       [[1.0000000e+00],
        [6.3636363e-01],
        [9.6666664e-01],
        ...,
        [3.3333334e-01],
        [6.5384614e-01],
        [1.0088916e+00]],

       [[1.0000000e+00],
        [6.3636363e-01],
        [1.0000001e+00],
        ...,
        [5.0000000e-01],
        [6.5384614e-01],
        [1.0094843e+00]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2013-01-01 2013-01-02 ... 2017

In [28]:
print(time_cov_val)

<TimeSeries (DataArray) (time: 16, component: 7, sample: 1)>
array([[[2.017e+03],
        [8.000e+00],
        [1.600e+01],
        [2.280e+02],
        [2.000e+00],
        [3.300e+01],
        [1.688e+03]],

       [[2.017e+03],
        [8.000e+00],
        [1.700e+01],
        [2.290e+02],
        [3.000e+00],
        [3.300e+01],
        [1.689e+03]],

       [[2.017e+03],
        [8.000e+00],
        [1.800e+01],
        [2.300e+02],
...
        [2.410e+02],
        [1.000e+00],
        [3.500e+01],
        [1.701e+03]],

       [[2.017e+03],
        [8.000e+00],
        [3.000e+01],
        [2.420e+02],
        [2.000e+00],
        [3.500e+01],
        [1.702e+03]],

       [[2.017e+03],
        [8.000e+00],
        [3.100e+01],
        [2.430e+02],
        [3.000e+00],
        [3.500e+01],
        [1.703e+03]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2017-08-16 2017-08-17 ... 2017-08-31
  * component  (component) object 'year' 'month' ... 'linear_increa

##Oil : moving average 활용해서, test 기간 동안의 oil 값 예측할 것.

- Moving Average : 데이터 값들 정규화 중 일종 **(날 별로 편차 최소화하기 위해 주변 날들과의 평균값으로)**
- window =3 인 Moving Average 적용 \\
[1, 2, 3, 4, 5]  -> [1,(1+2)/2, (1+2+3)/3, (2+3+4)/3, (3+4+5)/3] = [1, 1.5, 2, 3, 4]

In [29]:
from darts.models import MovingAverage

oil = TimeSeries.from_dataframe(df_oil,
                                time_col='date',
                                value_cols=['dcoilwtico'],
                                freq='D')

oil = oil.astype(np.float32)

#Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverage(window=7)
oil_moving_average_28 = MovingAverage(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")

ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")

#하나의 index에 두개의 값을 쌓는 다는 말
oil_moving_averages = ma_7.stack(ma_28)

In [30]:
#100번째 timestamp 에 대한 7th window average, 28th window average -> 얼추 값이 비슷한 모습
display(oil_moving_averages[100])

<TimeSeries (DataArray) (date: 1, component: 2, sample: 1)>
array([[[0.78416604],
        [0.78240633]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2013-04-11
  * component  (component) object 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [31]:
len(oil_moving_averages)

1704

##Holidays

- 누군가의 도움으로, 이렇게 holiday 를 간단하게 카테고리화 할 수 있게 되었답니다.
- 각 store 의 위치 및 지역을 고려하여 각 store 별로 우리는 해당 store 가 공휴일을 맞는지 안 맞는지 구분할 수 있음

In [32]:
def holiday_list(df_stores):

    listofseries = []
    
    for i in range(0,len(df_stores)):
            
            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = df_holidays_events["date"]
            
            #np.where : 조건에 맞는 index 찾기 -> 조건에 맞으면 '1', 조건에 맞지 않으면 '0' 으로 값 변경 

            #옆에 조건에 맞으면 'national_holiday' feature 에 '1' 아니면 '0' - one-hot encoding 진행하는 느낌 
            df_holiday_dummies["national_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & (df_holidays_events["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(df_holidays_events['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(df_holidays_events['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(df_holidays_events['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((df_holidays_events["type"] == "Event") & (df_holidays_events["locale"] == "National") & (~df_holidays_events['description'].str.contains('Terremoto Manabi')) & (~df_holidays_events['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((df_holidays_events["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((df_holidays_events["type"] == "Holiday") & ((df_holidays_events["locale_name"] == df_stores['state'][i]) | (df_holidays_events["locale_name"] == df_stores['city'][i]))), 1, 0)
                     
            listofseries.append(df_holiday_dummies)

    return listofseries

- listofseries 에는 가게별 df_holiday_dumies 총 54개가 들어가있다. listofseries[0] ~ listofseries[53]
- df_holiday_dumies (pd.DataFrame) : Feature 은 총 8개로, 공휴일을 분류해서 one-hot coding 시켜 놓은 느낌 -> (Date, national_holiday, earthquake_relief, .... ) 해당 되는 것에 '1' 박혀 있고, 아니면 '0' 




In [33]:
def remove_0_and_duplicates(holiday_list):

    listofseries = []
    
    for i in range(0,len(holiday_list)):
            
            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')

            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]
            
            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max', 
                                   'christmas':'max', 'football_event':'max', 
                                   'national_event':'max', 'work_day':'max', 
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries

In [34]:
def holiday_TS_list_54(holiday_list):

    listofseries = []
    
    for i in range(0,54):
            
            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i], 
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')
            
            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries

In [35]:
list_of_holidays_per_store = holiday_list(df_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)   
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)

In [36]:
display(len(holidays_transformed))
display(holidays_transformed[0].components.values)
display(holidays_transformed[0][100])

54

array(['national_holiday', 'earthquake_relief', 'christmas',
       'football_event', 'national_event', 'work_day', 'local_holiday'],
      dtype=object)

<TimeSeries (DataArray) (date: 1, component: 7, sample: 1)>
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 2013-04-11
  * component  (component) object 'national_holiday' ... 'local_holiday'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

> holidats_transformed 에는 각 스토어별로 시계열로 날짜별로 해당되는 holidays 들이 one-hot encoding 되어 들어있음.

> holidays_transformed[0][100] : 0번째 store 의 100번째 timestamp 에 해당하는 공휴일 = [0,0,0,0,0,0,0] : 어떤 공휴일도 해당이 안됨

##Promotion 
- test 셋에도 있는 feature 이기 때문에 한번에 묶어서 하겠습니다

In [37]:
df_promotion = pd.concat([df_train, df_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

family_promotion_dict = {}

for family in family_list:
  df_family = df_promotion.loc[df_promotion['family'] == family]

  list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                #두개를 묶어서 본다는 뜻 
                                group_cols=["store_nbr","family"],
                                value_cols="onpromotion",
                                fill_missing_dates=True,
                                freq='D')
  
  for ts in list_of_TS_promo:
    ts = ts.astype(np.float32)

  family_promotion_dict[family] = list_of_TS_promo

In [38]:
display(family_promotion_dict['AUTOMOTIVE'][0])

<TimeSeries (DataArray) (date: 1704, component: 1, sample: 1)>
array([[[0.]],

       [[0.]],

       [[0.]],

       ...,

       [[0.]],

       [[0.]],

       [[0.]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 'onpromotion'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\ncomponent  ...
    hierarchy:          None

> family_promotion_dict['AUTOMOTIVE'][0] : 0번쨰 store 의 'AUTOMOTIVE' 중 행사중인 상품 개수 -> [0,0,...0] (1704,1) = 1704일 동안 serial data

In [39]:
from tqdm import tqdm

promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
    promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
    promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

    promo_pipeline = Pipeline([promo_filler,
                              promo_scaler])
    
    promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])
    
    # Moving Averages for Promotion Family Dictionaries
    promo_moving_average_7 = MovingAverage(window=7)
    promo_moving_average_28 = MovingAverage(window=28)

    promotion_covs = [] 

    for ts in promotion_transformed:
        ma_7 = promo_moving_average_7.filter(ts)
        ma_7 = TimeSeries.from_series(ma_7.pd_series())  
        ma_7 = ma_7.astype(np.float32)
        ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
        ma_28 = promo_moving_average_28.filter(ts)
        ma_28 = TimeSeries.from_series(ma_28.pd_series())  
        ma_28 = ma_28.astype(np.float32)
        ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
        promo_and_mas = ts.stack(ma_7).stack(ma_28)
        promotion_covs.append(promo_and_mas)

    promotion_transformed_dict[key] = promotion_covs

100%|██████████| 33/33 [01:34<00:00,  2.86s/it]


In [40]:
promotion_transformed_dict.keys()

dict_keys(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE', 'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES', 'SEAFOOD'])

> 얘도 기름값과 마찬가지로, moving average 통해서 미래값 예측

> 근데. test set (2017-08-16 ~ 2017-08-31) 에 있는데 굳이 해야해? 왜? / 생각해보니까, oil price 도 test 기간 동안 있긴 함...ㅎ

In [41]:
display(promotion_transformed_dict['AUTOMOTIVE'][0].components.values)
display(promotion_transformed_dict['AUTOMOTIVE'][0][1])

array(['onpromotion', 'promotion_ma_7', 'promotion_ma_28'], dtype=object)

<TimeSeries (DataArray) (date: 1, component: 3, sample: 1)>
array([[[0.],
        [0.],
        [0.]]])
Coordinates:
  * date       (date) datetime64[ns] 2013-01-02
  * component  (component) object 'onpromotion' ... 'promotion_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\nglobal_comp...
    hierarchy:          None

> Onpromotion, 7일 예측, 28일 예측값 스택

##Grouping the covariates : future covariates 들을 하나의 Timeseries로 모아야 해

In [42]:
general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

In [43]:
#찢어준 날짜 component 7개 + oil componenet 3개 = 10개 -> (1704,10)
general_covariates

<TimeSeries (DataArray) (time: 1704, component: 10, sample: 1)>
array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.7929646 ],
        [0.79240197],
        [0.796154  ]],

       [[0.        ],
        [0.        ],
        [0.03333334],
        ...,
        [0.7929646 ],
        [0.7925303 ],
        [0.7960361 ]],

       [[0.        ],
        [0.        ],
        [0.06666668],
        ...,
...
        ...,
        [0.2400805 ],
        [0.24278495],
        [0.25032222]],

       [[1.        ],
        [0.6363636 ],
        [0.96666664],
        ...,
        [0.23415849],
        [0.24149394],
        [0.25014064]],

       [[1.        ],
        [0.6363636 ],
        [1.0000001 ],
        ...,
        [0.2495558 ],
        [0.24079117],
        [0.25054285]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 'year' 'month' ... 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [44]:
store_covariates_future = []

for store in range(0,len(store_list)):
  #store 별로 holidays 에다가 위에서 만든 general_covariates 쌓아줌 
  stacked_covariates = holidays_transformed[store].stack(general_covariates)
  store_covariates_future.append(stacked_covariates)


> 그러면 store_covariates_future 에는 store 별로 holidays, oil, date 시계열 특성들이 다 한번에 담겨있는거야. 

> store_covariates_future[0] : 0 번째 store 의 날짜별 oil, date, holiday 특성 (1704, 3+7+holiday 개수)

In [45]:
future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]

  future_covariates_dict[key] = covariates_future

100%|██████████| 33/33 [00:08<00:00,  3.88it/s]


In [46]:
display(future_covariates_dict['AUTOMOTIVE'][0].components)

Index(['onpromotion', 'promotion_ma_7', 'promotion_ma_28', 'national_holiday',
       'earthquake_relief', 'christmas', 'football_event', 'national_event',
       'work_day', 'local_holiday', 'year', 'month', 'day', 'dayofyear',
       'dayofweek', 'weekofyear', 'linear_increase', 'dcoilwtico', 'oil_ma_7',
       'oil_ma_28'],
      dtype='object', name='component')



> future_covariates_dict['AUTOMOTIVE'][0] 에 있는 Feature : (promotion 관련, holiday 관련, date 관련, oil 관련) = 0 번째 store 의 'AUTOMOTIVE" 항목에 대한 시계열 데이터

> 'onpromotion', 'promotion_ma_7', 'promotion_ma_28', 'national_holiday',
       'earthquake_relief', 'christmas', 'football_event', 'national_event',
       'work_day', 'local_holiday', 'year', 'month', 'day', 'dayofyear',
       'dayofweek', 'weekofyear', 'linear_increase', 'dcoilwtico', 'oil_ma_7',
       'oil_ma_28']



##Transactions- Past Covriates
: 모델 시작 전, 각 스토어별로 transaction 을 넣어주고 normalize 시키자

In [47]:
df_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                df_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

from datetime import datetime, timedelta

transactions_list_full = []

for ts in transactions_list:
  if ts.start_time() > pd.Timestamp('20130101'):
    end_time = (ts.start_time() - timedelta(days=1))
    delta = end_time - pd.Timestamp('20130101')
    zero_series = TimeSeries.from_times_and_values(
                              times=pd.date_range(start=pd.Timestamp('20130101'), 
                              end=end_time, freq="D"),
                              values=np.zeros(delta.days+1))
    ts = zero_series.append(ts)
    ts = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions")
    transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

In [48]:
display(transactions_transformed[0])

<TimeSeries (DataArray) (time: 1688, component: 1, sample: 1)>
array([[[0.        ]],

       [[0.69831293]],

       [[0.60635131]],

       ...,

       [[0.13761164]],

       [[0.57327158]],

       [[0.5600397 ]]])
Coordinates:
  * time       (time) datetime64[ns] 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 'transactions'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

> transactions_transformed : Store 별로 시계열 Transaction 값

#4. Model ( 드디어!! )


##단일 LightGBMModel

- family_tranformed_TS_dict (dictionary) : (33 [key=family] , 54, 1688)
  - value 는 sales, static_covariates 들로 여러 개들이 있음
- future_covariates_dict (dictionary) : (33 [key = family] , 54, 1704, 20)
- transactions_transformed (list) : ( 54, 1688 )
- family_list = ['AUTOMOTIVE', "BABY CARE', ,,, 'SEAFOOD']


In [49]:
print(family_list)

['AUTOMOTIVE' 'BABY CARE' 'BEAUTY' 'BEVERAGES' 'BOOKS' 'BREAD/BAKERY'
 'CELEBRATION' 'CLEANING' 'DAIRY' 'DELI' 'EGGS' 'FROZEN FOODS' 'GROCERY I'
 'GROCERY II' 'HARDWARE' 'HOME AND KITCHEN I' 'HOME AND KITCHEN II'
 'HOME APPLIANCES' 'HOME CARE' 'LADIESWEAR' 'LAWN AND GARDEN' 'LINGERIE'
 'LIQUOR,WINE,BEER' 'MAGAZINES' 'MEATS' 'PERSONAL CARE' 'PET SUPPLIES'
 'PLAYERS AND ELECTRONICS' 'POULTRY' 'PREPARED FOODS' 'PRODUCE'
 'SCHOOL AND OFFICE SUPPLIES' 'SEAFOOD']


In [50]:
#1. LightGBM 모델 구조 및 training
from darts.models import LightGBMModel

LGBM_Models_Submission = {}

display("Training...")

for family in tqdm(family_list):

  #sales_family : (54,1688,1) 왜 바꾼거야? 밑에 결과값 똑같은데? 
  sales_family = family_TS_transformed_dict[family]

  #training_data : (54,1688,1) = (store, time, sales) with static_covariates (정적 feature 들도 드러나지는 않지만, 포함되어 있음)
  training_data = [ts for ts in sales_family] 

  #TCN_covariates : (54,1704, 20) = (store, time, feature)
  TCN_covariates = future_covariates_dict[family]

  #train_sliced (54,1688,1) -> TCN_covariates[i] 와 time index 가 일치하는 training_data[i] 만 뽑아준 것
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

  #모델 구조 
  LGBM_Model_Submission = LightGBMModel(lags = 63,
                                        lags_future_covariates = (14,1),
                                        lags_past_covariates = [-16,-17,-18,-19,-20,-21,-22],
                                        output_chunk_length=1,
                                        random_state=2022,
                                        gpu_use_dp= "true",
                                        )
  #모델 학습
  LGBM_Model_Submission.fit(series=train_sliced, 
                        future_covariates=TCN_covariates,
                        past_covariates=transactions_transformed)

  #LGBM_Models_Submission : 품목(family) 별 예측 모델 
  LGBM_Models_Submission[family] = LGBM_Model_Submission

'Training...'

 21%|██        | 7/33 [02:50<10:35, 24.43s/it]


KeyboardInterrupt: ignored

In [ ]:
display(LGBM_Models_Submission)

In [ ]:
#2. LightGBM Prediction 
display("Predictions...")

LGBM_Forecasts_Families_Submission = {}

for family in tqdm(family_list):
  #위와 동일
  sales_family = family_TS_transformed_dict[family]
  #위와 동일
  training_data = [ts for ts in sales_family]
  #TCN_covariates 과 동일
  LGBM_covariates = future_covariates_dict[family]
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

  #LGBM_Models_Submission['AUTOMOTIVE'] : 'AUTOMOTIVE' 에 대한 모델
  #forecast_LGMB : 이제 예측 결과값인 것 같은데? - 학습 데이터랑 똑같네? 원래 트리는 그래..
  #n=16 : train_sliced (1688) 이후, 16개에 대한 예측   
  forecast_LGBM = LGBM_Models_Submission[family].predict(n=16,
                                         series=train_sliced,
                                         future_covariates=LGBM_covariates,
                                         past_covariates=transactions_transformed)
  
  LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

In [ ]:
#LGBM_Forecasts_Families_Submission['AUTOMOTIVE'] : 'AUTOMOTIVE' 품목에 대한 가게별 예측값  (54,1)
print(len(LGBM_Forecasts_Families_Submission['AUTOMOTIVE']))
display(LGBM_Forecasts_Families_Submission['AUTOMOTIVE'])

In [ ]:
#
LGBM_Forecasts_Families_back_Submission = {}

for family in tqdm(family_list):
  #inverse_transform 으로 정규화된 값 실제 값으로 풀어줌
  #LGBM_Forecasts_Families_back_Submission['AUTOMOTIVE'] : (54,1)
  LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

In [ ]:
#2017.08.16 - 2017.08.31 기간 동안 sales 예측
display(LGBM_Forecasts_Families_back_Submission['AUTOMOTIVE'])

In [ ]:
#54 = 가게 수
print(len(LGBM_Forecasts_Families_back_Submission['AUTOMOTIVE']))
#16 = train set (1688) 시점 이후, 16일에 대한 예측
print(len(LGBM_Forecasts_Families_back_Submission['AUTOMOTIVE'][0]))

In [ ]:
for family in tqdm(LGBM_Forecasts_Families_back_Submission):
  for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
    if (family_TS_dict[family][n].univariate_values()[-21:] == 0).all():
        LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)

listofseries = []

for store in tqdm(range(0,54)):
  for family in family_list:
      oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
      oneforecast.columns = ['fcast']
      listofseries.append(oneforecast)

df_forecasts = pd.concat(listofseries) 
df_forecasts.reset_index(drop=True, inplace=True)

# No Negative Forecasts
df_forecasts[df_forecasts < 0] = 0

#결과값을 submission form 에 맞춰주는 과정 
forecasts_kaggle = pd.concat([df_test_sorted, df_forecasts.set_index(df_test_sorted.index)], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle.sort_values(by=['id'])
forecasts_kaggle_sorted = forecasts_kaggle_sorted.drop(['date','store_nbr','family'], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle_sorted.rename(columns={"fcast": "sales"})
forecasts_kaggle_sorted = forecasts_kaggle_sorted.reset_index(drop=True)

# Submission
submission_kaggle = forecasts_kaggle_sorted

In [ ]:
submission_kaggle.head()

In [ ]:
# 결과 1 : 단일 LightGBM model 결과값
from datetime import datetime
dt = datetime.now()
result = dt.strftime("%Y%m%d%H%M%S")
string = result+'.csv'
submission_kaggle.to_csv(string, index=False)

#XGBoost 단일 모델

In [ ]:
#1. XGBoost 모델 구조 및 training
from darts.models import XGBModel

XGB_Models_Submission = {}

display("Training...")

for family in tqdm(family_list):

  #sales_family : (54,1688,1) 왜 바꾼거야? 밑에 결과값 똑같은데? 
  sales_family = family_TS_transformed_dict[family]

  #training_data : (54,1688,1) = (store, time, sales) with static_covariates (정적 feature 들도 드러나지는 않지만, 포함되어 있음)
  training_data = [ts for ts in sales_family] 

  #TCN_covariates : (54,1704, 20) = (store, time, feature)
  TCN_covariates = future_covariates_dict[family]

  #train_sliced (54,1688,1) -> TCN_covariates[i] 와 time index 가 일치하는 training_data[i] 만 뽑아준 것
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

  #모델 구조 
  XGB_Model_Submission = XGBModel(lags = 63,
                                        lags_future_covariates = (14,1),
                                        lags_past_covariates = [-16,-17,-18,-19,-20,-21,-22],
                                        output_chunk_length=1,
                                        random_state=2022,
                                        gpu_use_dp= "true",
                                        )
  #모델 학습
  XGB_Model_Submission.fit(series=train_sliced, 
                        future_covariates=TCN_covariates,
                        past_covariates=transactions_transformed)

  #LGBM_Models_Submission : 품목(family) 별 예측 모델 
  XGB_Models_Submission[family] = XGB_Model_Submission

'Training...'

  0%|          | 0/33 [00:00<?, ?it/s]

[16:21:49] WARNING: ../src/learner.cc:767: 
Parameters: { "gpu_use_dp" } are not used.



In [ ]:
#2. XGB Prediction 
display("Predictions...")

XGB_Forecasts_Families_Submission = {}

for family in tqdm(family_list):
  #위와 동일
  sales_family = family_TS_transformed_dict[family]
  #위와 동일
  training_data = [ts for ts in sales_family]
  #TCN_covariates 과 동일
  XGB_covariates = future_covariates_dict[family]
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

  #LGBM_Models_Submission['AUTOMOTIVE'] : 'AUTOMOTIVE' 에 대한 모델
  #forecast_LGMB : 이제 예측 결과값인 것 같은데? - 학습 데이터랑 똑같네? 원래 트리는 그래..
  #n=16 : train_sliced (1688) 이후, 16개에 대한 예측   
  forecast_XGB = XGB_Models_Submission[family].predict(n=16,
                                         series=train_sliced,
                                         future_covariates=LGBM_covariates,
                                         past_covariates=transactions_transformed)
  
  XGB_Forecasts_Families_Submission[family] = forecast_XGB

In [ ]:
#
XGB_Forecasts_Families_back_Submission = {}

for family in tqdm(family_list):
  #inverse_transform 으로 정규화된 값 실제 값으로 풀어줌
  #LGBM_Forecasts_Families_back_Submission['AUTOMOTIVE'] : (54,1)
  XGB_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(XGB_Forecasts_Families_Submission[family], partial=True)

In [ ]:
for family in tqdm(XGB_Forecasts_Families_back_Submission):
  for n in range(0,len(XGB_Forecasts_Families_back_Submission[family])):
    if (family_TS_dict[family][n].univariate_values()[-21:] == 0).all():
        XGB_Forecasts_Families_back_Submission[family][n] = XGB_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)

listofseries = []

for store in tqdm(range(0,54)):
  for family in family_list:
      oneforecast = XGB_Forecasts_Families_back_Submission[family][store].pd_dataframe()
      oneforecast.columns = ['fcast']
      listofseries.append(oneforecast)

df_forecasts = pd.concat(listofseries) 
df_forecasts.reset_index(drop=True, inplace=True)

# No Negative Forecasts
df_forecasts[df_forecasts < 0] = 0

#결과값을 submission form 에 맞춰주는 과정 
forecasts_kaggle = pd.concat([df_test_sorted, df_forecasts.set_index(df_test_sorted.index)], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle.sort_values(by=['id'])
forecasts_kaggle_sorted = forecasts_kaggle_sorted.drop(['date','store_nbr','family'], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle_sorted.rename(columns={"fcast": "sales"})
forecasts_kaggle_sorted = forecasts_kaggle_sorted.reset_index(drop=True)

# Submission
submission_kaggle_XGB = forecasts_kaggle_sorted

In [ ]:
submission_kaggle_XGB.head()

##LightGBM - Ensemble Model

In [ ]:
model_params = [
    {"lags" : 7, "lags_future_covariates" : (16,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 365, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 730, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}
]

In [ ]:
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from lightgbm import early_stopping

submission_kaggle_list = []

for params in model_params:

  LGBM_Models_Submission = {}

  display("Training...")

  for family in tqdm(family_list):

    # Define Data for family
    sales_family = family_TS_transformed_dict[family]
    training_data = [ts for ts in sales_family] 
    TCN_covariates = future_covariates_dict[family]
    train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

    LGBM_Model_Submission = LightGBMModel(lags = params["lags"],
                                          lags_future_covariates = params["lags_future_covariates"],
                                          lags_past_covariates = params["lags_past_covariates"],
                                          output_chunk_length=1,
                                          random_state=2022,
                                          gpu_use_dp= "false")
      
    LGBM_Model_Submission.fit(series=train_sliced, 
                          future_covariates=TCN_covariates,
                          past_covariates=transactions_transformed)

    LGBM_Models_Submission[family] = LGBM_Model_Submission
    
  display("Predictions...")


  LGBM_Forecasts_Families_Submission = {}

  for family in tqdm(family_list):

    sales_family = family_TS_transformed_dict[family]
    training_data = [ts for ts in sales_family]
    LGBM_covariates = future_covariates_dict[family]
    train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

    forecast_LGBM = LGBM_Models_Submission[family].predict(n=16,
                                          series=train_sliced,
                                          future_covariates=LGBM_covariates,
                                          past_covariates=transactions_transformed)
    
    LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

  # Transform Back

  LGBM_Forecasts_Families_back_Submission = {}

  for family in tqdm(family_list):

    LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

  # Prepare Submission in Correct Format

  for family in tqdm(LGBM_Forecasts_Families_back_Submission):
    for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
      if (family_TS_dict[family][n].univariate_values()[-21:] == 0).all():
          LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)
          
  listofseries = []

  for store in tqdm(range(0,54)):
    for family in family_list:
        oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
        oneforecast.columns = ['fcast']
        listofseries.append(oneforecast)

  df_forecasts = pd.concat(listofseries) 
  df_forecasts.reset_index(drop=True, inplace=True)

  # No Negative Forecasts
  df_forecasts[df_forecasts < 0] = 0
  forecasts_kaggle = pd.concat([df_test_sorted, df_forecasts.set_index(df_test_sorted.index)], axis=1)
  forecasts_kaggle_sorted = forecasts_kaggle.sort_values(by=['id'])
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.drop(['date','store_nbr','family'], axis=1)
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.rename(columns={"fcast": "sales"})
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.reset_index(drop=True)

  # Submission
  submission_kaggle_list.append(forecasts_kaggle_sorted)

In [ ]:
#결과값 2 : 앙상블 모델 결과
df_sample_submission['sales'] = (submission_kaggle[['sales']]+submission_kaggle_list[0][['sales']]+submission_kaggle_list[1][['sales']]+submission_kaggle_list[2][['sales']])/4

In [ ]:
df_sample_submission.head()

In [ ]:
df_sample_submission.to_csv('./Ensemble_result.csv', index=False)